# Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  4. Task 4: Evaluating our Pipeline with Ragas
  5. Task 6: Making Adjustments and Re-Evaluating

But first! Let's set some dependencies!

## Dependencies and API Keys:

> NOTE: Please skip the pip install commands if you are running the notebook locally.

In [47]:
!pip install -qU ragas==0.2.10

In [48]:
!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

We'll also need to provide our API keys.

First, OpenAI's for our LLM/embedding model combination!

In [49]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

Please enter your OpenAI API key!··········


OPTIONALLY:

We can also provide a Ragas API key - which you can sign-up for [here](https://app.ragas.io/).

In [50]:
os.environ["RAGAS_APP_TOKEN"] = getpass("Please enter your Ragas API key!")

Please enter your Ragas API key!··········


## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [51]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [52]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31413    0 31413    0     0  38530      0 --:--:-- --:--:-- --:--:-- 38496


In [53]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70272    0 70272    0     0  65151      0 --:--:--  0:00:01 --:--:-- 65187


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [54]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [55]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [56]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [57]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Was the best currently available LLM trained i...,[Prompt driven app generation is a commodity a...,The context poses the question of whether the ...,single_hop_specifc_query_synthesizer
1,How does Google's Gemini 1.5 Flash model compa...,"[gets you OpenAI’s most expensive model, o1. G...",Google's Gemini 1.5 Flash model is significant...,single_hop_specifc_query_synthesizer
2,Could you elaborate on the challenges associat...,[feed with the model and talk about what you c...,OpenAI initially started with a WebSocket API ...,single_hop_specifc_query_synthesizer
3,what Hugging Face do?,[dependent on AGI itself. A model that’s robus...,"Hugging Face hosts more than 1,000 models that...",single_hop_specifc_query_synthesizer
4,How does the training data impact the ethics o...,[<1-hop>\n\nCode may be the best application T...,The training data is crucial in determining th...,multi_hop_abstract_query_synthesizer
5,What are the challenges associated with the bl...,[<1-hop>\n\nCode may be the best application T...,The black box nature of Large Language Models ...,multi_hop_abstract_query_synthesizer
6,How have the training costs and environmental ...,[<1-hop>\n\nCode may be the best application T...,The training costs for large language models (...,multi_hop_abstract_query_synthesizer
7,How do the black box nature of AI and the ease...,[<1-hop>\n\nCode may be the best application T...,"The black box nature of AI, particularly in th...",multi_hop_abstract_query_synthesizer
8,How did the introduction of GPT-4 and its feat...,[<1-hop>\n\nfeed with the model and talk about...,The introduction of GPT-4 in 2023 marked a sig...,multi_hop_specific_query_synthesizer
9,How has Google's Gemini 1.5 Pro contributed to...,[<1-hop>\n\nPrompt driven app generation is a ...,"Google's Gemini 1.5 Pro, released in February,...",multi_hop_specific_query_synthesizer


#### OPTIONAL:

If you've provided your Ragas API key - you can use this web interface to look at the created data!

In [58]:
dataset.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/c550ca1d-36a5-4e08-ba12-902ea8f4b849


'https://app.ragas.io/dashboard/alignment/testset/c550ca1d-36a5-4e08-ba12-902ea8f4b849'

## LangChain RAG

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [59]:
path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [60]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

74

#### ❓ Question:

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

The chunk_overlap parameter in the RecursiveCharacterTextSplitter allows us to to maintain semantic relationship between the retrieved contexts to give a better AI response.

For example, consider the sentence:
"Jack went up the hill to fetch a bucket of water."

If we set chunk_size=6 and chunk_overlap=2 (using word counts for illustration), the text might be split as follows:

• Chunk 1: "Jack went up the hill"
• Chunk 2: "up the hill to fetch"
• Chunk 3: "to fetch a bucket"
• Chunk 4: "a bucket of water"

Because each chunk overlaps with the previous one by two words, the context at the boundaries is preserved. This means that if a user asks, "Why did Jack go up the hill?" the overlapping segments ensure that the necessary context (e.g., "went up the hill") is present in more than one chunk, thereby improving the chances of retrieving the correct answer.

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [61]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [62]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="ai_across_years",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ai_across_years",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [63]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [64]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Now we can produce a node for retrieval!

In [65]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### Augmented

Let's create a simple RAG prompt!

In [66]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-mini` to avoid using the same model as our judge model.

In [67]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Then we can create a `generate` node!

In [68]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [69]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [70]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [71]:
response = graph.invoke({"question" : "How are LLM agents useful?"})

In [72]:
response["response"]

'LLM agents can be useful in several ways:\n\n1. **Ease of Construction**: Building LLMs is surprisingly straightforward, requiring only a few hundred lines of Python code, as long as sufficient quality training data is available. This accessibility means that a wider range of individuals and organizations can experiment with and create LLMs.\n\n2. **Local Execution**: Recent advancements have made it possible to run LLMs on personal devices, reducing the barrier to access and enabling more users to leverage these models without needing expensive servers.\n\n3. **Code Generation**: LLMs have shown exceptional capability in writing code. The structured nature of programming languages makes them easier for LLMs to handle compared to natural languages, which contributes to their effectiveness in this domain.\n\n4. **Iterative Improvement**: LLMs can generate code and, with tools like ChatGPT Code Interpreter, are capable of executing that code, identifying errors, and making corrections. 

## Evaluating the App with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [73]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [74]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,Was the best currently available LLM trained i...,[Was the best currently available LLM trained ...,[Prompt driven app generation is a commodity a...,"Not quite, but almost! The context indicates t...",The context poses the question of whether the ...,single_hop_specifc_query_synthesizer
1,How does Google's Gemini 1.5 Flash model compa...,[I wrote about this at the time in The killer ...,"[gets you OpenAI’s most expensive model, o1. G...",Google's Gemini 1.5 Flash model is positioned ...,Google's Gemini 1.5 Flash model is significant...,single_hop_specifc_query_synthesizer
2,Could you elaborate on the challenges associat...,[These abilities are just a few weeks old at t...,[feed with the model and talk about what you c...,The initial implementation of the WebSocket AP...,OpenAI initially started with a WebSocket API ...,single_hop_specifc_query_synthesizer
3,what Hugging Face do?,[You can even run them entirely in your browse...,[dependent on AGI itself. A model that’s robus...,Hugging Face is a platform that facilitates th...,"Hugging Face hosts more than 1,000 models that...",single_hop_specifc_query_synthesizer
4,How does the training data impact the ethics o...,"[Since then, almost every major LLM (and most ...",[<1-hop>\n\nCode may be the best application T...,The training data used for large language mode...,The training data is crucial in determining th...,multi_hop_abstract_query_synthesizer
5,What are the challenges associated with the bl...,[Another common technique is to use larger mod...,[<1-hop>\n\nCode may be the best application T...,The challenges associated with the black box n...,The black box nature of Large Language Models ...,multi_hop_abstract_query_synthesizer
6,How have the training costs and environmental ...,"[If you can gather the right data, and afford ...",[<1-hop>\n\nCode may be the best application T...,The training costs and environmental impact of...,The training costs for large language models (...,multi_hop_abstract_query_synthesizer
7,How do the black box nature of AI and the ease...,"[Since then, almost every major LLM (and most ...",[<1-hop>\n\nCode may be the best application T...,The black box nature of AI and the ease of bui...,"The black box nature of AI, particularly in th...",multi_hop_abstract_query_synthesizer
8,How did the introduction of GPT-4 and its feat...,[This is a huge advantage for open over closed...,[<1-hop>\n\nfeed with the model and talk about...,The introduction of GPT-4 and its features in ...,The introduction of GPT-4 in 2023 marked a sig...,multi_hop_specific_query_synthesizer
9,How has Google's Gemini 1.5 Pro contributed to...,[I wrote about this at the time in The killer ...,[<1-hop>\n\nPrompt driven app generation is a ...,Google's Gemini 1.5 Pro has significantly adva...,"Google's Gemini 1.5 Pro, released in February,...",multi_hop_specific_query_synthesizer


Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [75]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [76]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

Next up - we simply evaluate on our desired metrics!

In [77]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

{'context_recall': 0.6416, 'faithfulness': 0.6936, 'factual_correctness': 0.4583, 'answer_relevancy': 0.8737, 'context_entity_recall': 0.2615, 'noise_sensitivity_relevant': 0.2580}

## Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!

> NOTE: This will be using Cohere's Rerank model (which was updated fairly [recently](https://docs.cohere.com/v2/changelog/rerank-v3.5)) - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

In [78]:
os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

Please enter your Cohere API key!··········


In [79]:
!pip install -qU cohere langchain_cohere


We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [80]:
retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [81]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [82]:
class State(TypedDict):
  question: str
  context: List[Document]
  response: str

graph_builder = StateGraph(State).add_sequence([retrieve_adjusted, generate])
graph_builder.add_edge(START, "retrieve_adjusted")
graph = graph_builder.compile()

In [83]:
response = graph.invoke({"question" : "How are LLM agents useful?"})
response["response"]

'LLM agents are seen as potentially useful in various ways, particularly in the realm of writing code. Their effectiveness in programming stems from the simpler grammar rules of programming languages compared to natural languages, making them adept at generating code. However, the broader utility of LLM agents is called into question due to challenges like gullibility; they struggle to distinguish truth from fiction, which raises concerns about their reliability in making meaningful decisions on behalf of users. Despite excitement around AI agents, there have been few successful implementations in production, primarily due to these trust issues. Overall, while LLMs have notable capabilities, especially in coding, their utility is tempered by significant skepticism regarding their decision-making abilities and the potential negative implications of their use.'

In [84]:
import time

for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(2) # To try to avoid rate limiting.

In [85]:
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[71]: TimeoutError()


{'context_recall': 0.6249, 'faithfulness': 0.7367, 'factual_correctness': 0.4375, 'answer_relevancy': 0.8740, 'context_entity_recall': 0.2864, 'noise_sensitivity_relevant': 0.3033}

#### ❓ Question:

Which system performed better, on what metrics, and why?

Before Re-Ranking:

**{'context_recall': 0.6416, 'faithfulness': 0.6936, 'factual_correctness': 0.4583, 'answer_relevancy': 0.8737, 'context_entity_recall': 0.2615, 'noise_sensitivity_relevant': 0.2580}**

After Re-Ranking:
**{'context_recall': 0.6249, 'faithfulness': 0.7367, 'factual_correctness': 0.4375, 'answer_relevancy': 0.8740, 'context_entity_recall': 0.2864, 'noise_sensitivity_relevant': 0.3033}**

As clearly seen the 'faithfulness' has improved a lot which means our LLM is hallucinating much lesser than the previous retriever. This is probably beacause of the availablity of much more contexts than what was previously available with the previous retriever. We have 20 retrieved contexts that have been compressed into 5 and therefore we see that the semantic relationships are much better maintained leading to better quality answers and lesser hallucinations after Re-Ranking.
